In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# selection
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel

# models 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



import time



In [2]:
train_df = pd.read_csv("option_train.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   1680 non-null   float64
 1   S       1680 non-null   float64
 2   K       1680 non-null   int64  
 3   tau     1680 non-null   float64
 4   r       1680 non-null   float64
 5   BS      1680 non-null   object 
dtypes: float64(4), int64(1), object(1)
memory usage: 78.9+ KB


## Preprocessing

In [3]:
train_df.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420,0.341270,0.03013,Under
1,0.125000,427.015526,465,0.166667,0.03126,Over
2,20.691244,427.762336,415,0.265873,0.03116,Under
3,1.035002,451.711658,460,0.063492,0.02972,Over
4,39.553020,446.718974,410,0.166667,0.02962,Under


In [4]:
train_df = train_df.drop(columns = ['BS']) 
# assume we can't use Value so we drop it

X = train_df.iloc[:, 1:]
y = train_df.iloc[:, 0]

In [5]:
X

,S,K,tau,r
0,431.623898,420,0.341270,0.03013
1,427.015526,465,0.166667,0.03126
2,427.762336,415,0.265873,0.03116
3,451.711658,460,0.063492,0.02972
4,446.718974,410,0.166667,0.02962
...,...,...,...,...
1675,444.397163,475,0.273810,0.03034
1676,438.453825,410,0.301587,0.02972
1677,428.042219,390,0.170635,0.03188
1678,439.081203,480,0.293651,0.02962


## Creating variables & MinMaxScale

In [6]:
# define function 
# multiplying all fields 

def create_variables(df):
    cols = list(df.columns)
    name = ''
    name2 = ''
    result = pd.Series(1, index=range(X.shape[0]))
    for i in cols:
        name = name + str(i) + "*"
        name2 = name2 + str(i) + "/"
        #result = result*X[i]
    #df[name] = result

    for i in cols:
        cols.remove(i)
        for ii in cols:
            if i != ii:
                name = str(i)+"*"+str(ii)
                name2 = str(i)+"/"+str(ii)
                df[name] = df[i]*df[ii]
                df[name2] = df[i]/df[ii]
        for iii in cols:
            #print(f'i:{i}, ii:{ii}, iii:{iii}')
            if i != iii and ii != iii and i != ii:
                #print(f'i:{i}, ii:{ii}, iii:{iii}')
                name = str(i)+"*"+str(ii)+"*"+str(iii)
                df[name] = df[i]*df[ii]*df[iii]

    return df

create_variables(X)

,S,K,tau,r,S*K,S/K,S*tau,S/tau,S*r,S/r,S*r*K,S*r*tau,tau*K,tau/K,tau*r,tau/r,tau*r*K
0,431.623898,420,0.341270,0.03013,181282.037286,1.027676,147.300219,1264.758401,13.004828,14325.386601,5462.027783,4.438156,143.333333,0.000813,0.010282,11.326580,4.318633
1,427.015526,465,0.166667,0.03126,198562.219543,0.918313,71.169254,2562.093150,13.348505,13660.125589,6207.054983,2.224751,77.500000,0.000358,0.005210,5.331627,2.422650
2,427.762336,415,0.265873,0.03116,177521.369606,1.030753,113.730463,1608.897145,13.329074,13727.931207,5531.565877,3.543841,110.337302,0.000641,0.008285,8.532510,3.438110
3,451.711658,460,0.063492,0.02972,207787.362588,0.981982,28.680105,7114.458665,13.424870,15198.911770,6175.440416,0.852373,29.206349,0.000138,0.001887,2.136341,0.868013
4,446.718974,410,0.166667,0.02962,183154.779504,1.089558,74.453163,2680.313841,13.231816,15081.666928,5425.044569,2.205303,68.333333,0.000407,0.004937,5.626829,2.024033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,444.397163,475,0.273810,0.03034,211088.652662,0.935573,121.680176,1623.015726,13.483010,14647.236767,6404.429722,3.691777,130.059524,0.000576,0.008307,9.024704,3.946006
1676,438.453825,410,0.301587,0.02972,179766.068250,1.069400,132.232106,1453.820576,13.030848,14752.820491,5342.647548,3.929938,123.650794,0.000736,0.008963,10.147621,3.674902
1677,428.042219,390,0.170635,0.03188,166936.465410,1.097544,73.038950,2508.526487,13.645986,13426.669354,5321.934517,2.328482,66.547619,0.000438,0.005440,5.352413,2.121538
1678,439.081203,480,0.293651,0.02962,210758.977248,0.914753,128.936544,1495.249499,13.005585,14823.808325,6242.680906,3.819100,140.952381,0.000612,0.008698,9.913936,4.175010


In [7]:
X['K-S'] = X.K - X.S

cols = X.columns
X = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(X, columns = cols )
X.head()

,S,K,tau,r,S*K,S/K,S*tau,S/tau,S*r,S/r,S*r*K,S*r*tau,tau*K,tau/K,tau*r,tau/r,tau*r*K,K-S
0,0.202299,0.36,0.867347,0.261603,0.336600,0.527326,0.829308,0.001031,0.243026,0.498204,0.280827,0.820816,0.749947,0.836173,0.858454,0.876363,0.742367,0.403330
1,0.050749,0.72,0.418367,0.738397,0.604273,0.102610,0.395640,0.012539,0.516604,0.153673,0.686948,0.406738,0.401345,0.363717,0.430090,0.407063,0.412605,0.854009
2,0.075309,0.32,0.673469,0.696203,0.278347,0.539275,0.638083,0.004083,0.501137,0.188789,0.318733,0.653510,0.575226,0.657345,0.689737,0.657636,0.589221,0.392988
3,0.862901,0.68,0.153061,0.088608,0.747172,0.349871,0.153608,0.052920,0.577393,0.950592,0.669715,0.149997,0.145619,0.134424,0.149465,0.156928,0.142213,0.584228
4,0.698712,0.28,0.418367,0.046414,0.365609,0.767650,0.414346,0.013587,0.423716,0.889872,0.260667,0.403100,0.352805,0.413739,0.407007,0.430172,0.343275,0.175348


In [8]:
X_copy = X
X_origin = X.iloc[:,0:4]

In [9]:
X.shape

(1680, 18)

## Feature Selection
- Kbest

In [10]:
# build importance by coefficient 

# select by importance 

In [11]:
from sklearn.linear_model import LassoCV

lasso = LassoCV().fit(X,y)
importance = np.abs(lasso.coef_)
feature_names = np.array(X.columns)

# choose 5 
threshold = np.sort(importance)[-6]+0.01

sfm = SelectFromModel(lasso, threshold=threshold).fit(X, y)


In [12]:
X = X.loc[:,list(feature_names[sfm.get_support()])]
X

,S*K,S/K,S*r,S*r*K,K-S
0,0.336600,0.527326,0.243026,0.280827,0.403330
1,0.604273,0.102610,0.516604,0.686948,0.854009
2,0.278347,0.539275,0.501137,0.318733,0.392988
3,0.747172,0.349871,0.577393,0.669715,0.584228
4,0.365609,0.767650,0.423716,0.260667,0.175348
...,...,...,...,...,...
1675,0.798310,0.169640,0.623674,0.794539,0.786949
1676,0.313117,0.689362,0.263739,0.215752,0.250435
1677,0.114385,0.798663,0.753408,0.204461,0.163327
1678,0.793203,0.088783,0.243629,0.706368,0.880667


## Model

In [13]:
kf = KFold(n_splits = 10, shuffle = True)

x_dict = {
    'X': X,
    'X_copy': X_copy,
    'X_origin': X_origin
}

### baseline: linear regression

In [14]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

cross_val_score(model, X, y, scoring='r2', cv=kf).mean()

0.9575448466715081

### knn


In [15]:
%%time
from sklearn.neighbors import KNeighborsRegressor
table = pd.DataFrame(columns=['n_neighbors', 'weights','X_set', 'r2'])

row_index = 0
for i in [3, 4, 5]:
    for ii in ['uniform', 'distance']:
        for x in ['X', 'X_copy', 'X_origin']:
            table.loc[row_index, 'n_neighbors'] = i
            table.loc[row_index, 'weights'] = ii
            table.loc[row_index, 'X_set'] = x

            model = KNeighborsRegressor(
                n_neighbors = i,
                weights = ii
            )

            r2 = round(cross_val_score(model, x_dict[x], y, scoring='r2', cv=kf).mean(), 4)
            table.loc[row_index, 'r2'] = r2
            print(f'Row {row_index} calculation finished.')
            row_index += 1

table.sort_values("r2", ascending=False)

Row 0 calculation finished.
Row 1 calculation finished.
Row 2 calculation finished.
Row 3 calculation finished.
Row 4 calculation finished.
Row 5 calculation finished.
Row 6 calculation finished.
Row 7 calculation finished.
Row 8 calculation finished.
Row 9 calculation finished.
Row 10 calculation finished.
Row 11 calculation finished.
Row 12 calculation finished.
Row 13 calculation finished.
Row 14 calculation finished.
Row 15 calculation finished.
Row 16 calculation finished.
Row 17 calculation finished.
CPU times: user 1.5 s, sys: 27.8 ms, total: 1.53 s
Wall time: 1.07 s


,n_neighbors,weights,X_set,r2
16,5,distance,X_copy,0.9863
10,4,distance,X_copy,0.9863
4,3,distance,X_copy,0.9848
7,4,uniform,X_copy,0.9831
13,5,uniform,X_copy,0.983
1,3,uniform,X_copy,0.9825
11,4,distance,X_origin,0.978
17,5,distance,X_origin,0.977
5,3,distance,X_origin,0.9749
14,5,uniform,X_origin,0.9731


### DT

In [16]:
from sklearn.tree import DecisionTreeRegressor
# max_depth [3, 5]
# which hyperparameters?

### NN

In [17]:
%%time
from sklearn.neural_network import MLPRegressor

table = pd.DataFrame(columns=['hidden_layer_sizes', 'solver', 'max_iter','X_set', 'Error'])

row_index = 0
for i in [(100,3),(100,5),(200,3)]:
    for ii in ['sgd', 'adam']:
        for iii in [150, 200]:
            for x in ['X', 'X_copy', 'X_origin']:
                table.loc[row_index, 'hidden_layer_sizes'] = str(i)
                table.loc[row_index, 'solver'] = ii
                table.loc[row_index, 'max_iter'] = iii
                table.loc[row_index, 'X_set'] = x

                model = MLPRegressor(hidden_layer_sizes = i, solver = ii, max_iter=iii)

                r2 = round(cross_val_score(model, x_dict[x], y, scoring='r2', cv=kf).mean(), 4)
                table.loc[row_index, 'r2'] = r2
                print(f'Row {row_index} calculation finished.')
                row_index += 1

table.sort_values("r2", ascending=False)

Row 0 calculation finished.
Row 1 calculation finished.
Row 2 calculation finished.
Row 3 calculation finished.
Row 4 calculation finished.
Row 5 calculation finished.
Row 6 calculation finished.
Row 7 calculation finished.
Row 8 calculation finished.
Row 9 calculation finished.
Row 10 calculation finished.
Row 11 calculation finished.
Row 12 calculation finished.
Row 13 calculation finished.
Row 14 calculation finished.
Row 15 calculation finished.
Row 16 calculation finished.
Row 17 calculation finished.
Row 18 calculation finished.
Row 19 calculation finished.
Row 20 calculation finished.
Row 21 calculation finished.
Row 22 calculation finished.
Row 23 calculation finished.
Row 24 calculation finished.
Row 25 calculation finished.
Row 26 calculation finished.
Row 27 calculation finished.
Row 28 calculation finished.
Row 29 calculation finished.
Row 30 calculation finished.
Row 31 calculation finished.
Row 32 calculation finished.
Row 33 calculation finished.
Row 34 calculation finis

,hidden_layer_sizes,solver,max_iter,X_set,Error,r2
16,"(100, 5)",sgd,200,X_copy,NaN,0.9992
13,"(100, 5)",sgd,150,X_copy,NaN,0.9992
17,"(100, 5)",sgd,200,X_origin,NaN,0.9989
19,"(100, 5)",adam,150,X_copy,NaN,0.9979
34,"(200, 3)",adam,200,X_copy,NaN,0.9969
22,"(100, 5)",adam,200,X_copy,NaN,0.9957
23,"(100, 5)",adam,200,X_origin,NaN,0.9931
8,"(100, 3)",adam,150,X_origin,NaN,0.9799
3,"(100, 3)",sgd,200,X,NaN,0.9675
18,"(100, 5)",adam,150,X,NaN,0.9587
